In [1]:
import os
import json
import h5py
import numpy as np

import robomimic
import robomimic.utils.file_utils as FileUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.obs_utils as ObsUtils
import imageio
import tqdm
from robomimic.utils.file_utils import create_hdf5_filter_key

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/ns1254/miniconda3/envs/equidiff/lib/python3.9/site-packages/robomimic/scripts/setup_macros.py
)


In [2]:
# dataset_path = "/home/ns1254/data_franka/bellpepper_oma/ola_114.hdf5"
dataset_path = "/home/ns1254/robomimic/datasets/can/mh/image_v141.hdf5"

# dataset_path = "/home/ns1254/robomimic/datasets/square/mh/image_v141.hdf5"

f_org = h5py.File(dataset_path, "r")
demos = list(f_org["data"].keys())

In [3]:
lengths=[]
for demo_name in demos:
    demo=f_org['data'][demo_name]
    num_samples=demo.attrs['num_samples']
    lengths.append(num_samples)

lengths=np.array(lengths)

print('Number of demos: ', len(demos))
print('Max length: ', np.max(lengths))
print('Min length: ', np.min(lengths))
print('Mean length: ', np.mean(lengths))

Number of demos:  300
Max length:  1050
Min length:  98
Mean length:  209.18666666666667


In [4]:
f_org['mask'].keys()

<KeysViewHDF5 ['20_percent', '20_percent_train', '20_percent_valid', '50_percent', '50_percent_train', '50_percent_valid', 'better', 'better_operator_1', 'better_operator_1_train', 'better_operator_1_valid', 'better_operator_2', 'better_operator_2_train', 'better_operator_2_valid', 'better_train', 'better_valid', 'okay', 'okay_better', 'okay_better_train', 'okay_better_valid', 'okay_operator_1', 'okay_operator_1_train', 'okay_operator_1_valid', 'okay_operator_2', 'okay_operator_2_train', 'okay_operator_2_valid', 'okay_train', 'okay_valid', 'train', 'valid', 'worse', 'worse_better', 'worse_better_train', 'worse_better_valid', 'worse_okay', 'worse_okay_train', 'worse_okay_valid', 'worse_operator_1', 'worse_operator_1_train', 'worse_operator_1_valid', 'worse_operator_2', 'worse_operator_2_train', 'worse_operator_2_valid', 'worse_train', 'worse_valid']>

In [5]:
def copy_group(src_group, dest_group):
    # Copy attributes
    for attr_name, attr_value in src_group.attrs.items():
        dest_group.attrs[attr_name] = attr_value

    for key, item in src_group.items():
        if isinstance(item, h5py.Group):
            # Create a new group in the destination and recursively copy contents
            new_group = dest_group.create_group(key)
            copy_group(item, new_group)
        elif isinstance(item, h5py.Dataset):
            # Copy datasets
            dataset = dest_group.create_dataset(key, data=item[...])
            # Copy attributes for the dataset
            for attr_name, attr_value in item.attrs.items():
                dataset.attrs[attr_name] = attr_value


def copy_demos(f_src, f_dest, demos2copy, next_id):
    new_demo_names=[]
    i=next_id
    for demo_name in tqdm.tqdm(demos2copy):
        demo=f_src['data'][demo_name]  

        demo_name_new="demo_"+str(i) 
        new_demo=f_dest["data"].create_group(demo_name_new) 
        copy_group(demo, new_demo) 

        new_demo_names.append(demo_name_new)
        i = i+1

    return i , new_demo_names

In [6]:
# demos

In [7]:
can_30_ids = []
for i in range(300):
    if i%10==1:
        can_30_ids.append(i)
can_30_ids = np.array(can_30_ids)
can_30_ids.shape

can_100_ids=[]
for i in range(300):
    if i%3==1:
        can_100_ids.append(i)
can_100_ids = np.array(can_100_ids)
can_100_ids.shape

(100,)

In [8]:
# demos2copy = [f"demo_{i}" for i in can_30_ids]
demos2copy = [f"demo_{i}" for i in can_100_ids]
demos2copy

['demo_1',
 'demo_4',
 'demo_7',
 'demo_10',
 'demo_13',
 'demo_16',
 'demo_19',
 'demo_22',
 'demo_25',
 'demo_28',
 'demo_31',
 'demo_34',
 'demo_37',
 'demo_40',
 'demo_43',
 'demo_46',
 'demo_49',
 'demo_52',
 'demo_55',
 'demo_58',
 'demo_61',
 'demo_64',
 'demo_67',
 'demo_70',
 'demo_73',
 'demo_76',
 'demo_79',
 'demo_82',
 'demo_85',
 'demo_88',
 'demo_91',
 'demo_94',
 'demo_97',
 'demo_100',
 'demo_103',
 'demo_106',
 'demo_109',
 'demo_112',
 'demo_115',
 'demo_118',
 'demo_121',
 'demo_124',
 'demo_127',
 'demo_130',
 'demo_133',
 'demo_136',
 'demo_139',
 'demo_142',
 'demo_145',
 'demo_148',
 'demo_151',
 'demo_154',
 'demo_157',
 'demo_160',
 'demo_163',
 'demo_166',
 'demo_169',
 'demo_172',
 'demo_175',
 'demo_178',
 'demo_181',
 'demo_184',
 'demo_187',
 'demo_190',
 'demo_193',
 'demo_196',
 'demo_199',
 'demo_202',
 'demo_205',
 'demo_208',
 'demo_211',
 'demo_214',
 'demo_217',
 'demo_220',
 'demo_223',
 'demo_226',
 'demo_229',
 'demo_232',
 'demo_235',
 'demo_23

In [9]:
# square_50_ids = []
# for i in range(300):
#     if i%6==1:
#         square_50_ids.append(i)
# square_50_ids = np.array(square_50_ids)
# print(square_50_ids.shape)

# square_150_ids=[]
# for i in range(300):
#     if i%2==1:
#         square_150_ids.append(i)
# square_150_ids = np.array(square_150_ids)
# print(square_150_ids.shape)

In [10]:
# demos2copy = [f"demo_{i}" for i in square_50_ids]
# demos2copy = [f"demo_{i}" for i in square_150_ids]
# demos2copy

### create sub data

In [11]:
group_name = "bad"
# demos2copy =[demo.decode('utf-8') for demo in f_org['mask'][group_name]]
# demos2copy=sorted(demos2copy, key=lambda x: int(x.split('_')[-1]))
len(demos2copy), demos2copy

(100,
 ['demo_1',
  'demo_4',
  'demo_7',
  'demo_10',
  'demo_13',
  'demo_16',
  'demo_19',
  'demo_22',
  'demo_25',
  'demo_28',
  'demo_31',
  'demo_34',
  'demo_37',
  'demo_40',
  'demo_43',
  'demo_46',
  'demo_49',
  'demo_52',
  'demo_55',
  'demo_58',
  'demo_61',
  'demo_64',
  'demo_67',
  'demo_70',
  'demo_73',
  'demo_76',
  'demo_79',
  'demo_82',
  'demo_85',
  'demo_88',
  'demo_91',
  'demo_94',
  'demo_97',
  'demo_100',
  'demo_103',
  'demo_106',
  'demo_109',
  'demo_112',
  'demo_115',
  'demo_118',
  'demo_121',
  'demo_124',
  'demo_127',
  'demo_130',
  'demo_133',
  'demo_136',
  'demo_139',
  'demo_142',
  'demo_145',
  'demo_148',
  'demo_151',
  'demo_154',
  'demo_157',
  'demo_160',
  'demo_163',
  'demo_166',
  'demo_169',
  'demo_172',
  'demo_175',
  'demo_178',
  'demo_181',
  'demo_184',
  'demo_187',
  'demo_190',
  'demo_193',
  'demo_196',
  'demo_199',
  'demo_202',
  'demo_205',
  'demo_208',
  'demo_211',
  'demo_214',
  'demo_217',
  'demo_

In [12]:
for key in f_org['data'].attrs.keys():
    print(key)

env_args
total


In [13]:
dataset_path_sub =dataset_path
dataset_path_sub=dataset_path_sub.replace(".hdf5", "")+"_can100.hdf5"
dataset_path_sub

'/home/ns1254/robomimic/datasets/can/mh/image_v141_can100.hdf5'

In [14]:
f_sub = h5py.File(dataset_path_sub, "w") 
f_sub.create_group("data")
f_sub.create_group("mask")

<HDF5 group "/mask" (0 members)>

In [15]:
#copy attributes of exist.
key='env_args'
f_sub['data'].attrs[key]=f_org['data'].attrs[key]

In [16]:
print('copying demos...') 
next_id , new_demo_names=copy_demos(f_src=f_org, f_dest=f_sub, demos2copy=demos2copy, next_id=0)

copying demos...


100%|██████████| 100/100 [00:02<00:00, 39.53it/s]


In [17]:
f_org.close()
f_sub.close()